# Goal of notebook + SQL Code

In this notebook, we will be working with the data with the names of plaintiff and defendant. If possible, we will try to get all the names of all plantiffs and defenders then see the distribution using groupby.\

Note: We can also do groupby with SQL.\

Example of SQL code that will be useful\

SELECT col_of_interest, count(col_of_interest)\
FROM name_of_table\
GROUP BY col_of_interest

# Import Libraries

In [165]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Explore Data analysis

In [166]:
df = pd.read_csv('name_of_plantiff_defender.csv')


What we have here is the defend for each example.\
Discover Bank is the plaintiff and Phan, Kevin T is the defendant

In [167]:
# first, we need to check if there are any missing data
df.isnull().values.any()

False

In [168]:
df.columns

Index(['post_id', 'text', 'last_indexed'], dtype='object')

In [169]:
df = df[['post_id', 'text']]

In [170]:
df.head(10)

,post_id,text
0,5464,"Potfolio Recovery Associates, Llc vs. Irby, JR..."
1,5469,"Discover Bank vs. Phan, Kevin T |"
2,5475,"Wilcox, Clarence vs. Mobile Gas Station Owner ..."
3,5481,"Rab Performance Recoveries, Llc vs. Ware, Levi |"
4,5487,"Midland Funding, Llc v. Norman, Pamela |"
5,5497,"Hamilton, Natasha R v. Yell-O-Glow Corporation |"
6,5505,"Capital One Bank (Usa), N.A. vs. George, Devah..."
7,5509,"Capital One Bank (Usa), N.A. vs. Divino, Mai H |"
8,5515,"Clean Way Waste Services, Inc vs. Justice Cafe..."
9,5522,Better Life Physical Therapy v. Commerce Insur...


Before we split into two columns, plaintiff and defendant. Notice some examples are vs. or v.\
This is problematic since we cant split the text using the delimiter 'vs.'\
Because we have vs. and v. in some title. We might have to split into two dataframe then merge together..

In [171]:
df2 = pd.DataFrame(df['text'].str.split('vs.', 1).tolist(),
                        columns = ['plaintiff', 'defendant'])

Notice that some defendant has None values?\
that means those title has 'v.' instead of 'vs.'\
We may need to filter out then create another dataframe.

In [172]:
# currently, those examples that didn't split well has None in defendant. We need to fillna with np.nan so pandas can filter them
df2 = df2.fillna(value=np.nan)

In [173]:
df2

,plaintiff,defendant
0,"Potfolio Recovery Associates, Llc","Irby, JR , Thomas |"
1,Discover Bank,"Phan, Kevin T |"
2,"Wilcox, Clarence",Mobile Gas Station Owner Akmal Moawed |
3,"Rab Performance Recoveries, Llc","Ware, Levi |"
4,"Midland Funding, Llc v. Norman, Pamela |",NaN
...,...,...
1048568,First Financial Portfolio Management Inc. v. Y...,NaN
1048569,"Attleboro Pawn Shop , Inc.","Ace Gutter Service, Inc. |"
1048570,Eastpointe Rehab & Nursing,"Marino, Joan |"
1048571,Leeway Realty Trust As officer of or agent for...,NaN


In [174]:
bad_examples = df2[df2['defendant'].isnull()]
bad_examples.head(5)

,plaintiff,defendant
4,"Midland Funding, Llc v. Norman, Pamela |",NaN
5,"Hamilton, Natasha R v. Yell-O-Glow Corporation |",NaN
9,Better Life Physical Therapy v. Commerce Insur...,NaN
18,"Nstar Electric Company v. Thompson, Anthony |",NaN
54,"Rockland Trust Company v. Overbea, Adgirene |",NaN


For bad examples, we need to split it again, so we get the results we wanted!!! Notice every examples have 'v.'

In [175]:
df3 = pd.DataFrame(bad_examples['plaintiff'].str.split('v.', 1).tolist(),
                        columns = ['plaintiff', 'defendant'])
df3.head(5)

,plaintiff,defendant
0,"Midland Funding, Llc","Norman, Pamela |"
1,"Hamilton, Natasha R",Yell-O-Glow Corporation |
2,Better Life Physical Therapy,Commerce Insurance Company |
3,Nstar Electric Company,"Thompson, Anthony |"
4,Rockland Trust Company,"Overbea, Adgirene |"


In [176]:
# notice we have a bar above? we will get rid of the bar
df3['defendant'] = df3['defendant'].str.replace('|','')


C:\Users\16178\AppData\Local\Temp\ipykernel_22288\2244843344.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df3['defendant'] = df3['defendant'].str.replace('|','')


In [177]:
df3.shape

(748705, 2)

perfect !! We able to split the results into two pieces as well. Now what's left is to merge again with the good_examples!

In [178]:
Good_examples = df2[~df2['defendant'].isnull()]
Good_examples.head(5)

# ~ serves as a negation for us. So what the above code means is that, we want rows that is not null

,plaintiff,defendant
0,"Potfolio Recovery Associates, Llc","Irby, JR , Thomas |"
1,Discover Bank,"Phan, Kevin T |"
2,"Wilcox, Clarence",Mobile Gas Station Owner Akmal Moawed |
3,"Rab Performance Recoveries, Llc","Ware, Levi |"
6,"Capital One Bank (Usa), N.A.","George, Devahn R |"


In [179]:
# samething we want to get rid of the bar...
Good_examples['defendant'] = Good_examples['defendant'].str.replace('|', '')


C:\Users\16178\AppData\Local\Temp\ipykernel_22288\5645497.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Good_examples['defendant'] = Good_examples['defendant'].str.replace('|', '')
C:\Users\16178\AppData\Local\Temp\ipykernel_22288\5645497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Good_examples['defendant'] = Good_examples['defendant'].str.replace('|', '')


In [180]:
Good_examples.shape

(299868, 2)

## Concating the two dataframes

In [181]:
final_df = pd.concat([Good_examples, df3])

In [182]:
final_df

,plaintiff,defendant
0,"Potfolio Recovery Associates, Llc","Irby, JR , Thomas"
1,Discover Bank,"Phan, Kevin T"
2,"Wilcox, Clarence",Mobile Gas Station Owner Akmal Moawed
3,"Rab Performance Recoveries, Llc","Ware, Levi"
6,"Capital One Bank (Usa), N.A.","George, Devahn R"
...,...,...
748700,"Signet Electronic Systems, Inc.","T.R. White Co., Inc."
748701,Disco,"r Bank v. Khan, Agha Naeem"
748702,First Financial Portfolio Management Inc.,"Young, Janice"
748703,Leeway Realty Trust As officer of or agent for...,"Baiona, Lawrence"


how can we check we have the final results concat? Well just add the shape of both dataframe. Which is 299868 + 748705 = 1048573

#  count number of plaintiff and defendant

Notice that you see some examples as Capital One Bank (USA), N.A. and Capital One Bank (Usa), N.A.?\
These are the samething, but the dataset comes in with different alphabet lower case and upperecase. We should lowercase every single examples then count them..\
Also we should replace comma with empty string. Some examples have comma in wrong position..

In [183]:
final_df['plaintiff'] = final_df['plaintiff'].str.upper()

In [184]:
final_df['plaintiff'].isnull().any()

False

We have no null for this.

In [185]:
count_plaintiff = pd.DataFrame(final_df['plaintiff'].value_counts())
count_plaintiff.head(8)

,plaintiff
"CAPITAL ONE BANK (USA), N.A.",29476
PORTFOLIO RECO,24924
DISCO,16795
"CACH, LLC",13849
CA,11905
MIDLAND FUNDING LLC,10030
DISCOVER BANK,10002
"MIDLAND FUNDING, LLC",9872


In [119]:
# save this to excel files
count_plaintiff.to_excel('num_Of_plaintiff.xlsx')

# Do the same for Defendant

In [187]:
final_df['defendant'] = final_df['defendant'].str.upper()


In [188]:
non_null_final_df = final_df.loc[final_df['defendant'].notnull()]

In [189]:
non_null_final_df.isnull().any()

plaintiff    False
defendant    False
dtype: bool

good now we dont have any missing in dataframe

In [190]:
count_defendant = pd.DataFrame(non_null_final_df['defendant'].value_counts())
count_defendant.head(8)

,defendant
COMMERCE INSURANCE COMPANY,2064
LIBERTY MUTUAL INSURANCE COMPANY,1023
,714
SAFETY INSURANCE COMPANY,592
ARBELLA MUTUAL INSURANCE COMPANY,369
METROPOLITAN PROPERTY & CASUALTY INSURANCE COMPANY,360
PLYMOUTH ROCK ASSURANCE CORPORATION,309
BOSTON HOUSING AUTHORITY,308


For some reason, we may run into illegalcharacter error. We will install xlsxwriter..

In [126]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


In [127]:
# save it as excel files
count_defendant.to_excel('defendant.xlsx', engine='xlsxwriter')

# Key Takeaway

Most defendant top are insurance companys\
Most plaintiff top are banks.